# Introduction

## This is the official counterparts evaluation script of CLS tasks
* Use google colab pro+ (high RAM+GPU)
* we use the P100 GPU for the Experiments

## The code and Training process along with all record are public avaliable
* Our github page: https://github.com/sagizty/Multi-Stage-Hybrid-Transformer
* The ROSE dataset is not publicly aviliable.
* The Warwick dataset is publicly aviliable, therefore we use it for illustration.


# 说明
* 这是一个公开的colab脚本，意在展现深度学习做病理学图像任务的能力。
* 本人的github会陆续更新更多自己的开源工作，包括模型设计（backbone），训练策略设计，预训练工作，数据增强工作等。欢迎持续关注，感谢您的每一个star！任何疑问可以去github联系我。
* 更多详细介绍会逐步更新



In [1]:
# check GPU
!nvidia-smi

Mon Jun 27 13:39:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!date --date='+8 hour'  # CST time zone

Mon Jun 27 21:39:36 UTC 2022


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## create file-system enviroment
* mount your google drive first


In [ ]:
# create file-system enviroment
# mount the google drive first
# https://drive.google.com/drive/u/1/my-drive

# clear colab path
!rm -rf /data
!rm -rf /home/MIL_Experiment

# create path
!mkdir /home/MIL_Experiment
!mkdir /home/MIL_Experiment/runs
!mkdir /home/MIL_Experiment/code
!mkdir /home/MIL_Experiment/saved_models
!mkdir /home/MIL_Experiment/imaging_results

!mkdir /data
!mkdir /data/MIL_Experiment
!mkdir /data/MIL_Experiment/dataset

print('Folder Tree Creation completed!')

# get the latest code from Github MIL-SI official page
!git clone https://github.com/sagizty/Multi-Stage-Hybrid-Transformer.git /home/MIL_Experiment/code
print('code transfer from github completed!')

# copy runs if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/runs/* /home/MIL_Experiment/runs
# print('tensorboard log transfer completed!')

# copy saved_models if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/saved_models/* /home/MIL_Experiment/saved_models
# print('saved_models transfer completed!')

# get the MIL and CLS dataset from github
# by its zip
# !cp /home/MIL_Experiment/code/sample_datasets/warwick_MIL.zip /data/MIL_Experiment/dataset/
!cp /home/MIL_Experiment/code/sample_datasets/warwick_CLS.zip /data/MIL_Experiment/dataset/
# unzip
# !unzip -q /data/MIL_Experiment/dataset/warwick_MIL.zip -d /data/MIL_Experiment/dataset/
!unzip -q /data/MIL_Experiment/dataset/warwick_CLS.zip -d /data/MIL_Experiment/dataset/

# alter the path
# !rm -rf /data/MIL_Experiment/dataset/warwick_MIL.zip
!rm -rf /data/MIL_Experiment/dataset/warwick_CLS.zip
print('data transfer completed!')

Folder Tree Creation completed!
Cloning into '/home/MIL_Experiment/code'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 451 (delta 4), reused 0 (delta 0), pack-reused 434
Receiving objects: 100% (451/451), 116.26 MiB | 41.80 MiB/s, done.
Resolving deltas: 100% (201/201), done.
code transfer from github completed!
data transfer completed!


## Arrange the working enviorment

In [ ]:
!pip install -q torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:09tcmalloc: large alloc 1147494400 bytes == 0x3a3d6000 @  0x7f3e8335a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 60.5 MB/s eta 0:00:17tcmalloc: large alloc 1434370048 bytes == 0x7ea2c000 @  0x7f3e8335a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.3 MB/s eta 0:08:51tcmalloc: large alloc 1792966656 bytes == 0x385e000 @  0x7f3e8335a615 0x592b76 0x4df71e 0x59afff 0x515655 0x

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code")
!pwd

# get packages
!pip install tensorboardX
!pip install timm
!pip install notifyemail
!pip install ttach


/home/MIL_Experiment/code
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip list

Package                       Version
----------------------------- ------------------------------
absl-py                       1.1.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.2
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis                          0.7.7
bokeh                         2.3.3
branca                        0.5.0
bs4                           0.0.1
CacheControl                  0.

## Start Training
* by command line
* use argparse to set down hyper-parameter

* 5-fold experiment is used here

# CLS counterparts

* Train

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=2, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_d

In [ ]:
!python Train.py --model_idx ResNet50_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='ResNet50_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=2, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:01<00:00, 57.5MB/s]
/usr/local/lib/python3.7

In [ ]:
!python Train.py --model_idx ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=2, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224']
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Name

In [ ]:
!python Train.py --model_idx efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=2, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficie

In [ ]:
!python Train.py --model_idx swin_b_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='swin_b_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=2, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_

# Test

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',
 'vit_gigantic_patch14_224',
 'vit_h

In [ ]:
!python Test.py --model_idx ResNet50_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
test model output： tensor([[0.8139, 0.0185]], grad_fn=<AddmmBackward>)
model is ready now!
model loaded
model : ResNet50_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=True, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, mod

In [ ]:
!python Test.py --model_idx ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

['vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224']
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
test model output： tensor([[ 0.2782, -0.6109]], grad_fn=<AddmmBackward>)
model is ready now!
model loaded
model : ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enab

In [ ]:
!python Test.py --model_idx efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',
 'efficientnet_cc_b0_4e',
 'efficientnet_cc_b0_8e',
 'efficientnet_cc_b1_8e',
 'efficientnet_el',
 'efficientnet_el_pruned',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_es_pruned',
 'efficientnet_l2',
 'efficientnet_lite0',
 'efficientnet_lite1',
 'efficientnet_lite2',
 'efficientnet_lite3',
 'efficientnet_lite4',
 'efficientnetv2_l',
 'efficientnetv2_m',
 'efficientnetv2_rw_m',
 'efficientnetv2_rw_s',
 'efficientnetv2_rw_t',
 'efficientnetv2_s',
 'efficientnetv2_xl',
 'gc_efficientnetv2_rw_t',
 'tf_efficientnet_b0',
 'tf_efficientnet_b0_ap',
 'tf_efficientnet_b0_ns',
 'tf_efficientnet_b1',
 'tf_efficientnet_b1_ap',
 'tf_efficientnet_b1_ns',
 'tf_efficientnet_b2',
 'tf_efficien

In [ ]:
!python Test.py --model_idx swin_b_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window7_224_in22k',
 'swin_large_patch4_window12_384',
 'swin_large_patch4_window12_384_in22k',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224']
test model output： tensor([[0.1134, 0.6528]], grad_fn=<AddmmBackward>)
model is ready now!
model loaded
model : swin_b_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=True, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='swin_b_384_401_PT_lf05_b4_warwick_C

# Synchronize files

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code/utils")
!python check_log_json.py --draw_root /home/MIL_Experiment/runs --record_dir /home/MIL_Experiment/CSV_logs

In [ ]:
# create path on google drive
!mkdir /content/drive/MyDrive/MIL_SI_sample
# copy the results
!/bin/cp -rf /home/MIL_Experiment/* /content/drive/MyDrive/MIL_SI_sample/
print('results copy completed!')

mkdir: cannot create directory ‘/content/drive/MyDrive/MIL_SI_sample’: File exists
results copy completed!


In [4]:
!date --date='+8 hour'  # CST time zone

Mon Jun 27 21:39:50 UTC 2022
